# Weather Joining
## Description
Here we demonstrate how our weather data was joined to our usage data for modelling.

## Imports

In [1]:
import json
import pandas as pd
from sqlalchemy import create_engine

## Database connection

In [2]:
# load credentials
credentials_file_path = '../credentials.json'
with open(credentials_file_path) as credentials_file:
    credentials = json.load(credentials_file)
    
# connect to database
engine = create_engine('mysql+mysqldb://{user}@{host}/{db}'.format(
    user = credentials['user'],
    host = credentials['host'],
    db = credentials['db']
))

conn = engine.connect()

## Remove tables if they exist
Used for refreshing the database.

In [3]:
# drop table if exists
conn.execute('DROP TABLE IF EXISTS model_data')

## Load the weather data

In [4]:
temps = pd.read_sql('''
    SELECT 
        city_state,
        year,
        month,
        day,
        AVG(T_HR_AVG) AS avg_temp,
        GROUP_CONCAT(T_HR_AVG) AS hourly_temps
    FROM temps
    GROUP BY city_state, year, month, day
''', conn)

# MySQL gives us the GROUP_CONCAT(...) result as a string so we need to back it out to an array of floats
temps['hourly_temps'] = temps['hourly_temps'].apply(lambda hourly_temps: [float(hourly_temp) for hourly_temp in hourly_temps.split(',')])

# drop any rows without 24 hours of temperatures
temps = temps[temps['hourly_temps'].apply(len) == 24].copy()

# output
temps

,city_state,year,month,day,avg_temp,hourly_temps
1,"Austin, Texas",2018,1,2,-4.083333,"[-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4...."
2,"Austin, Texas",2018,1,3,-1.116667,"[-4.4, -4.7, -4.8, -5.1, -5.4, -5.0, -5.5, -6...."
3,"Austin, Texas",2018,1,4,6.404167,"[9.5, 7.6, 7.3, 6.6, 5.2, 4.6, 4.6, 4.2, 4.4, ..."
4,"Austin, Texas",2018,1,5,8.308333,"[8.8, 7.8, 7.4, 6.6, 6.2, 6.3, 6.4, 6.2, 6.1, ..."
5,"Austin, Texas",2018,1,6,8.987500,"[12.1, 10.5, 9.7, 9.4, 8.9, 8.0, 7.4, 6.9, 6.5..."
...,...,...,...,...,...,...
360,"Austin, Texas",2018,12,27,13.925000,"[19.1, 19.1, 19.2, 19.2, 14.5, 10.5, 10.3, 10...."
361,"Austin, Texas",2018,12,28,6.733333,"[14.6, 12.7, 11.7, 10.2, 8.4, 7.4, 6.2, 5.2, 4..."
362,"Austin, Texas",2018,12,29,4.350000,"[7.5, 6.3, 5.2, 5.2, 5.1, 4.6, 4.2, 3.9, 3.8, ..."
363,"Austin, Texas",2018,12,30,2.991667,"[4.9, 4.7, 4.6, 4.3, 3.8, 3.4, 3.2, 3.0, 2.8, ..."


## Load the usage data

In [5]:
usage_daily = pd.read_sql('SELECT * FROM usage_daily', conn)
usage_daily

,dataid,dt,year,month,day,cooling,furnace,heaters,heating,solar,usage
0,661,2018-01-01 00:00:00-06,2018,1,1,0.000,346.957,0.0,346.957,651.007,1630.086
1,661,2018-01-02 00:00:00-06,2018,1,2,0.000,337.424,0.0,337.424,275.095,1006.282
2,661,2018-01-03 00:00:00-06,2018,1,3,-0.089,266.800,0.0,266.800,1865.626,-415.662
3,661,2018-01-04 00:00:00-06,2018,1,4,-0.055,221.171,0.0,221.171,1491.558,-254.565
4,661,2018-01-05 00:00:00-06,2018,1,5,-0.079,169.175,0.0,169.175,1581.860,-324.504
...,...,...,...,...,...,...,...,...,...,...,...
8665,9922,2018-12-27 00:00:00-06,2018,12,27,38.109,247.978,0.0,247.978,0.000,2814.867
8666,9922,2018-12-28 00:00:00-06,2018,12,28,-4.615,432.708,0.0,432.708,0.000,2825.281
8667,9922,2018-12-29 00:00:00-06,2018,12,29,-4.765,503.116,0.0,503.116,0.000,2992.122
8668,9922,2018-12-30 00:00:00-06,2018,12,30,-4.570,443.991,0.0,443.991,0.000,2975.587


In [6]:
usage_daily = usage_daily.set_index('dataid')

## Load the metadata
We will use this to get a location for each residence for joining with the weather data

In [7]:
# load the raw metadata csv file
metadata = pd.read_csv('../data/metadata.csv')

# output
metadata

,dataid,active_record,building_type,city,state,egauge_1min_min_time,egauge_1min_max_time,egauge_1min_data_availability,egauge_1s_min_time,egauge_1s_max_time,...,survey_2017,survey_2019,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,program_shines
0,The unique identifier for the home. To be more...,this field will show yes if this participant i...,This field will read Single-Family Home if thi...,participant's city,participant's state,"If eGauge electricity data is present, this fi...","If eGauge electricity data is present, this fi...",This field shows how much one minute data is ...,If eGauge electricity data is present in one s...,If eGauge electricity data is present in one s...,...,This field will show 'yes' if this participant...,This field will show 'yes' if this participant...,This field will show 'yes' if this participant...,This field will show 'yes' if this participant...,This field will show 'yes' if this participant...,This field will show 'yes' if this participant...,This field will show 'yes' if this participant...,This field will show one of the following grou...,This field will show one of the following grou...,This field will show 'yes' if this participant...
1,2836,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN
2,2743,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN
3,5323,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN
4,8560,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,891,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705,6601,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN
1706,114,yes,Single-Family Home 001 (Master),Austin,Texas,2013-10-15 19:00:00-05:00,2019-11-10 08:59:00-06:00,99%,2019-02-22 06:00:00-06:00,2019-11-10 08:00:00-06:00,...,NaN,NaN,NaN,yes,yes,NaN,NaN,CCET - Pricing Trial,NaN,NaN
1707,997,yes,Single-Family Home 001 (Master),Berkeley,California,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# get rid of the first description row 
metadata = metadata[1:].copy()

metadata['dataid'] = metadata['dataid'].astype(int)

# combine city and state into a single column for joining
metadata['city_state'] = metadata['city'] + ', ' + metadata['state']

# prepare for join, we only want the join key (dataid) and the city_state
metadata = metadata[['dataid', 'city_state']].set_index('dataid')

# output
metadata

,city_state
dataid,
2836,"Austin, Texas"
2743,"Austin, Texas"
5323,"Austin, Texas"
8560,"Austin, Texas"
3313,"Austin, Texas"
...,...
891,"Austin, Texas"
6601,"Austin, Texas"
114,"Austin, Texas"


## Join and save to database

In [9]:
usage_daily_w_metadata = usage_daily.join(metadata)
usage_daily_w_metadata

,dt,year,month,day,cooling,furnace,heaters,heating,solar,usage,city_state
dataid,,,,,,,,,,,
661,2018-01-01 00:00:00-06,2018,1,1,0.000,346.957,0.0,346.957,651.007,1630.086,"Austin, Texas"
661,2018-01-02 00:00:00-06,2018,1,2,0.000,337.424,0.0,337.424,275.095,1006.282,"Austin, Texas"
661,2018-01-03 00:00:00-06,2018,1,3,-0.089,266.800,0.0,266.800,1865.626,-415.662,"Austin, Texas"
661,2018-01-04 00:00:00-06,2018,1,4,-0.055,221.171,0.0,221.171,1491.558,-254.565,"Austin, Texas"
661,2018-01-05 00:00:00-06,2018,1,5,-0.079,169.175,0.0,169.175,1581.860,-324.504,"Austin, Texas"
...,...,...,...,...,...,...,...,...,...,...,...
9922,2018-12-27 00:00:00-06,2018,12,27,38.109,247.978,0.0,247.978,0.000,2814.867,"Austin, Texas"
9922,2018-12-28 00:00:00-06,2018,12,28,-4.615,432.708,0.0,432.708,0.000,2825.281,"Austin, Texas"
9922,2018-12-29 00:00:00-06,2018,12,29,-4.765,503.116,0.0,503.116,0.000,2992.122,"Austin, Texas"


In [10]:
usage_daily_w_metadata = usage_daily_w_metadata.reset_index().set_index(['city_state', 'year', 'month', 'day'])
usage_daily_w_metadata

dataid                      dt  cooling  \
city_state    year month day                                            
Austin, Texas 2018 1     1       661  2018-01-01 00:00:00-06    0.000   
                         2       661  2018-01-02 00:00:00-06    0.000   
                         3       661  2018-01-03 00:00:00-06   -0.089   
                         4       661  2018-01-04 00:00:00-06   -0.055   
                         5       661  2018-01-05 00:00:00-06   -0.079   
...                              ...                     ...      ...   
                   12    27     9922  2018-12-27 00:00:00-06   38.109   
                         28     9922  2018-12-28 00:00:00-06   -4.615   
                         29     9922  2018-12-29 00:00:00-06   -4.765   
                         30     9922  2018-12-30 00:00:00-06   -4.570   
                         31     9922  2018-12-31 00:00:00-06   -4.493   

                              furnace  heaters  heating     solar     usage  
city_state    year month day                                                 
Austin, Texas 2018 1     1    346.957      0.0  346.957   651.007  1630.086  
                         2    337.424      0.0  337.424   275.095  1006.282  
                         3    266.800      0.0  266.800  1865.626  -415.662  
                         4    221.171      0.0  221.171  1491.558  -254.565  
                         5    169.175      0.0  169.175  1581.860  -324.504  
...                               ...      ...      ...       ...       ...  
                   12    27   247.978      0.0  247.978     0.000  2814.867  
                         28   432.708      0.0  432.708     0.000  2825.281  
                         29   503.116      0.0  503.116     0.000  2992.122  
                         30   443.991      0.0  443.991     0.000  2975.587  
                         31   327.351      0.0  327.351     0.000  3125.235  

[8670 rows x 8 columns]

In [11]:
temps = temps.set_index(['city_state', 'year', 'month', 'day'])
temps

avg_temp  \
city_state    year month day              
Austin, Texas 2018 1     2    -4.083333   
                         3    -1.116667   
                         4     6.404167   
                         5     8.308333   
                         6     8.987500   
...                                 ...   
                   12    27   13.925000   
                         28    6.733333   
                         29    4.350000   
                         30    2.991667   
                         31    7.041667   

                                                                   hourly_temps  
city_state    year month day                                                     
Austin, Texas 2018 1     2    [-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4....  
                         3    [-4.4, -4.7, -4.8, -5.1, -5.4, -5.0, -5.5, -6....  
                         4    [9.5, 7.6, 7.3, 6.6, 5.2, 4.6, 4.6, 4.2, 4.4, ...  
                         5    [8.8, 7.8, 7.4, 6.6, 6.2, 6.3, 6.4, 6.2, 6.1, ...  
                         6    [12.1, 10.5, 9.7, 9.4, 8.9, 8.0, 7.4, 6.9, 6.5...  
...                                                                         ...  
                   12    27   [19.1, 19.1, 19.2, 19.2, 14.5, 10.5, 10.3, 10....  
                         28   [14.6, 12.7, 11.7, 10.2, 8.4, 7.4, 6.2, 5.2, 4...  
                         29   [7.5, 6.3, 5.2, 5.2, 5.1, 4.6, 4.2, 3.9, 3.8, ...  
                         30   [4.9, 4.7, 4.6, 4.3, 3.8, 3.4, 3.2, 3.0, 2.8, ...  
                         31   [3.6, 3.5, 3.3, 3.4, 3.3, 3.5, 3.5, 3.6, 3.9, ...  

[364 rows x 2 columns]

In [12]:
usage_daily_w_temps = usage_daily_w_metadata.join(temps)

# drop rows we didnt have temperature data for
usage_daily_w_temps = usage_daily_w_temps.dropna()

usage_daily_w_temps

dataid                      dt   cooling  \
city_state    year month day                                             
Austin, Texas 2018 1     2       661  2018-01-02 00:00:00-06     0.000   
                         2      1642  2018-01-02 00:00:00-06     2.474   
                         2      2335  2018-01-02 00:00:00-06     2.275   
                         2      2361  2018-01-02 00:00:00-06    47.886   
                         2      2818  2018-01-02 00:00:00-06     0.242   
...                              ...                     ...       ...   
                   12    31     8565  2018-12-31 00:00:00-06    38.654   
                         31     9019  2018-12-31 00:00:00-06     1.424   
                         31     9160  2018-12-31 00:00:00-06     0.408   
                         31     9278  2018-12-31 00:00:00-06  1205.246   
                         31     9922  2018-12-31 00:00:00-06    -4.493   

                              furnace  heaters  heating    solar     usage  \
city_state    year month day                                                 
Austin, Texas 2018 1     2    337.424      0.0  337.424  275.095  1006.282   
                         2      0.000      0.0    0.000  261.061   937.587   
                         2    475.461      0.0  475.461  236.227  1192.313   
                         2    419.124      0.0  419.124  254.814  -932.353   
                         2    399.285      0.0  399.285  243.497   741.889   
...                               ...      ...      ...      ...       ...   
                   12    31   439.949      0.0  439.949    0.000  1551.472   
                         31    49.241      0.0   49.241  264.470   820.896   
                         31   137.032      0.0  137.032  631.769    86.761   
                         31   208.127      0.0  208.127  793.183  2183.497   
                         31   327.351      0.0  327.351    0.000  3125.235   

                              avg_temp  \
city_state    year month day             
Austin, Texas 2018 1     2   -4.083333   
                         2   -4.083333   
                         2   -4.083333   
                         2   -4.083333   
                         2   -4.083333   
...                                ...   
                   12    31   7.041667   
                         31   7.041667   
                         31   7.041667   
                         31   7.041667   
                         31   7.041667   

                                                                   hourly_temps  
city_state    year month day                                                     
Austin, Texas 2018 1     2    [-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4....  
                         2    [-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4....  
                         2    [-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4....  
                         2    [-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4....  
                         2    [-3.8, -4.0, -3.8, -3.9, -4.0, -4.0, -4.1, -4....  
...                                                                         ...  
                   12    31   [3.6, 3.5, 3.3, 3.4, 3.3, 3.5, 3.5, 3.6, 3.9, ...  
                         31   [3.6, 3.5, 3.3, 3.4, 3.3, 3.5, 3.5, 3.6, 3.9, ...  
                         31   [3.6, 3.5, 3.3, 3.4, 3.3, 3.5, 3.5, 3.6, 3.9, ...  
                         31   [3.6, 3.5, 3.3, 3.4, 3.3, 3.5, 3.5, 3.6, 3.9, ...  
                         31   [3.6, 3.5, 3.3, 3.4, 3.3, 3.5, 3.5, 3.6, 3.9, ...  

[8645 rows x 10 columns]

In [13]:
# reset the index so all of the columns make it into the database
usage_daily_w_temps = usage_daily_w_temps.reset_index()

# dealing with array columns is a pain so let's treat it as a string
usage_daily_w_temps['hourly_temps'] = usage_daily_w_temps['hourly_temps'].astype(str)

# save dataframe to database
usage_daily_w_temps.to_sql('model_data', conn, index=False)